In [1]:
%cd '/content/'

/content


In [2]:
!rm -r '/content/pdf_parsing/'

In [3]:
!git clone https://github.com/waleedrazakhan92/pdf_parsing.git

Cloning into 'pdf_parsing'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [16]:
%cd '/content/pdf_parsing/'

/content/pdf_parsing


In [5]:
!mkdir utils

In [1]:
%cd '/content/pdf_parsing/utils/'

/content/pdf_parsing/utils


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!cp -r '/content/drive/MyDrive/shared_with_faizan/micheal_solomon/documents/' '/content/'
# !cp -r '/content/drive/MyDrive/shared_with_faizan/micheal_solomon/junk/page_tables/' '/content/'

In [ ]:
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!pip install pytesseract
!apt-get install poppler-utils
!pip install pdf2image
!pip install deskew
!pip install PyPDF2

In [1]:
%cd '/content/pdf_parsing/'

/content/pdf_parsing


#Functions

## Preprocessing Functions

In [2]:
%%writefile load_and_preprocess_utils.py
import pytesseract
from PIL import Image
import os
import re
import cv2
from tqdm import tqdm
from glob import glob
from PIL import Image, ImageChops

# for de skew
from deskew import determine_skew  ## gray image
import numpy as np
from skimage.transform import rotate

# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
# def cv2_imshow_rgb(img,resize=None):
#     if resize!=None:
#         img=cv2.resize(img,resize)


#     cv2_imshow(cv2.cvtColor(np.uint8(img),cv2.COLOR_RGB2BGR))

def cv2_imshow_rgb(img,resize=None, figsize=(15,15)):
    if resize!=None:
        img=cv2.resize(img,resize)

        #     cv2_imshow(cv2.cvtColor(np.uint8(img),cv2.COLOR_RGB2BGR))

    plt.figure(figsize=figsize)
    plt.imshow(np.uint8(img))


def display_multi(*images,resize=None, figsize=(15,15),bgr=False,axis=1):
    if resize!=None:
        res = np.array(cv2.resize(images[0],resize))
    else:
        res = np.array(images[0])

    for i in range(1,len(images)):

        if resize!=None:
            res_img = np.array(cv2.resize(images[i],resize))
        else:
            res_img = np.array(images[i])

        res = np.concatenate((res, res_img), axis=axis)

    if bgr==True:
        res = cv2.cvtColor(res,cv2.COLOR_BGR2RGB)

    return cv2_imshow_rgb(res,resize=None, figsize=figsize)

def load_img(img_path, rgb=True, size=False):
    img = cv2.imread(img_path)
    if rgb==True:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if size:
        img = cv2.resize(img, size)

    return img

def write_text_file(text_file_path,output_text,remove_empty_lines=True):
    if remove_empty_lines==True:
        output_text = output_text.split('\n')

    # Save the extracted text to a file
    with open(text_file_path, 'w') as f:
        if remove_empty_lines==True:
            for line in output_text:
                if line.strip():
                    f.write("%s\n" % line)
        else:
            f.write(output_text)


def deskew_image(image,cval=1.0):
    ## https://pypi.org/project/deskew/#:~:text=Deskewing%20is%20a%20process%20whereby,rather%20than%20at%20an%20angle.
    angle = determine_skew(image)
    image = rotate(image, angle, resize=True,cval=cval) * 255
    image = image.astype(np.uint8)
    return image


def add_image_border(img,border_width,border_color):
    img_border = cv2.copyMakeBorder(img,border_width,border_width,border_width,border_width,
                                    cv2.BORDER_CONSTANT,value=border_color)
    return img_border


def save_all_pages_raw(temp_path,all_pages,dpi=(300,300)):
    if all_pages:
        os.makedirs(temp_path, exist_ok=True)

        # Save images to the temporary directory
        for i in tqdm(range(0,len(all_pages))):
            image = all_pages[i]

            image_path = os.path.join(temp_path,"page_"+str(i+1)+".tiff")
            # image = Image.fromarray(image)
            image.save(image_path,dpi=dpi)
            image.save(image_path.split('.tiff')[0]+'.png',dpi=dpi)


def preprocess_image(image,deskew=True,img_thresh=False,add_border=False,border_width=10,border_color=(128,128,128),img_type='rgb'):
    assert img_type=='rgb' or img_type=='gray'

    # expects rgb image
    if img_type=='rgb':
        image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    if deskew==True:
        image = deskew_image(image,cval=1.0)

    if add_border==True:
        image = add_image_border(image,border_width,border_color)

    if img_thresh==True:
        ret,image = cv2.threshold(image,220,255,cv2.THRESH_BINARY)

    if img_type=='rgb':
        image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)

    return image


## rotate using tesseract
def check_orientation_tesseract(img):
    newdata = pytesseract.image_to_osd(np.array(img))
    orgi_orientation = re.search('(?<=Rotate: )\d+', newdata).group(0)
    # check image orientation
    if orgi_orientation != 0:
        return rotate_image_tesseract(img)
    else:
        return img

## pip install opevcv-python==4.1.0.25 (this functionality works only in v4.1.0.25 )
def rotate_image_tesseract(image, center=None, scale=1.0):
    angle = 360 - int(re.search('(?<=Rotate: )\d+', pytesseract.image_to_osd(image)).group(0))
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    mmm = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, mmm, (w, h))

    return rotated


def trim_image_borders(im,margin=10):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()

    if bbox:
        x,y,x_max,y_max = bbox
        x = max(x-margin,0)
        y = max(y-margin,0)
        x_max = min(x_max+margin,im.size[0]-1)
        y_max = min(y_max+margin,im.size[1]-1)
        bbox = (x,y,x_max,y_max)
        return im.crop(bbox)
    else:
        return im

Overwriting load_and_preprocess_utils.py


## All Cases

In [3]:
%%writefile notice_and_case_types.py
##############################################################
## Notice types( from Tables)
##############################################################

notice_types_tables = {}
## same lines i.e "Notice Type: XYZ"
notice_types_tables['Reopen'] = 'Reopen Notice'
notice_types_tables['Transfer'] = 'Transfer Notice'
notice_types_tables['USCIS'] = 'USCIS Account Access'
notice_types_tables['Approval'] = 'Approval Notice'
## Next lines i.e:
## Notice Type
## Receipt
notice_types_tables['Receipt'] = 'Receipt'
notice_types_tables['Rejection'] = 'Rejection Notice'

'''
Receipt with no "notice type" heading "same as ASC Appointment"  same as "Initial interview"
'''
notice_types_tables['Interview'] = 'Request for Applicant to Appear for '
notice_types_tables['ASC'] = 'ASC Appointment Notice'

# notice_types_tables['Receipt_2'] = 'Receipt'
notice_types_tables['Receipt_2'] = 'Recei' ## if it was cut by line

notice_types_tables['Interview_2'] = 'Please come to:'
notice_types_tables['Cancellation'] = 'Notice of Interview Cancellation'
notice_types_tables['Applicants'] = 'Notice to Applicants'
notice_types_tables['Biometric'] = 'Biometric'


##############################################################
## Document types
##############################################################
all_doc_tags = {}
all_doc_tags['Courtesy_letter'] = 'COURTESY LETTER TO APPLICANT'
all_doc_tags['NIVCC'] = 'NOTICE OF IMMIGRANT VISA CASE CREATION'
all_doc_tags['Withdrawal_Acknowledgment'] = 'ACKNOWLEDGMENT OF WITHDRAWAL'
all_doc_tags['Withdrawal'] = 'WITHDRAWAL'
all_doc_tags['Decision'] = 'DECISION'
all_doc_tags['Decision_Notice'] = 'NOTICE OF DECISION'
all_doc_tags['RFE'] = 'REQUEST FOR EVIDENCE'
# all_doc_tags['RFE_2'] = 'REQUEST FOR EVIDENCE (FORM I-485)'
# all_doc_tags['RFE_3'] = 'REQUEST FOR EVIDENCE (FORM 1-485)'
all_doc_tags['Deficiency_Notice'] = 'DEFICIENCY NOTICE'
# all_doc_tags['DefN_2'] = 'I-693 DEFICIENCY NOTICE'
# all_doc_tags['DefN_3'] = '1-693 DEFICIENCY NOTICE'
all_doc_tags['Oath_Ceremony'] = 'NOTICE OF NATURALIZATION'

## Partial Tags
all_doc_tags_partial = ['REQUEST FOR EVIDENCE','DEFICIENCY NOTICE','NOTICE OF NATURALIZATION']

##############################################################
## Case types
##############################################################
# ['I-129F','I-130','I-485','I-131','I-765','N-400','I-751','I-90','I-539']
all_case_types_org = ['-129F','-130','-485','-131','-765','-400','-751','-539','-290B','-90']
all_case_types_tables = ['129F','130','485','131','765','400','751','539','290B','90']


Overwriting notice_and_case_types.py


## Table Utils

In [4]:
%%writefile table_utils.py

from utils.common_functions import *
from utils.load_and_preprocess_utils import add_image_border
import pytesseract
import os
import cv2
#from pytesseract import Output
import numpy as np
import skimage
from utils.load_and_preprocess_utils import display_multi

def apply_morph(img,morph_operation=cv2.MORPH_CLOSE,kernel=(3,3),iterations=1):
    img = cv2.morphologyEx(img, morph_operation, kernel=kernel,iterations=iterations)
    return img

## case type in case of tables
def find_case_type_from_tables(extracted_text,all_case_types):
    '''
    finds tag lines in extracted text
    '''
    det_case_type = None
    for c in ['Case Type','CaseType']:
        c_t_idx = find_element_index(extracted_text,c)
        if c_t_idx!=[]:
            break


    if c_t_idx!=[] and len(extracted_text)>1:
        tag_line = extracted_text[c_t_idx+1]
        for in_tag in all_case_types:
            if in_tag in tag_line:
                return in_tag,tag_line
        else:
            return None,None
    else:
        return None,None



def test_cropped_to_text(in_crop):
    extracted_text = pytesseract.image_to_string(in_crop)
    print(extracted_text)
    if extracted_text != '\x0c':
        extracted_text = delete_empty_lines(extracted_text)

    return extracted_text
    # print('----------------------')
    # print(extracted_text)

def draw_fake_line(img,line_hei=0.5,line_thick=4):
    '''
    drawing a fake line in the middle of the document to
    cater for entries which didnt come inside a table at first i.e name, c/o, etc
    '''
    return cv2.line(img, (0,int(line_hei*img.shape[0])), (img.shape[1],int(line_hei*img.shape[0])), 0, line_thick)


def make_fake_table(img,table_coordinates,h_limit=0.35,w_limit=0.6):
    '''
    pre_reqs = other tables were found
    generate a fake table by finding the maximum y from old tables
    then from y_max to img_height*h_limit and 0:w_limit crops a fake table
    '''
    img_h,img_w = img.shape
    coordis_array = np.array(table_coordinates['cropped'])
    max_y_idx = np.argmax(coordis_array[:,5]) ## format = (x,y,w,h,x+w,y+h)
    max_y = coordis_array[:,5][max_y_idx]
    fake_table = img[max_y:int(img_h*h_limit), :int(img_w*w_limit)]

    ## appending fake_table_coordinates to table_coordinates too
    x,y,w,h = 0, max_y, int(img_w*w_limit), int(img_h*h_limit)-max_y
    table_coordinates['cropped'].append((x,y,w,h,x+w,y+h))

    return fake_table,table_coordinates


## Notice type
def find_notice_tags(all_tags,extracted_text):
    '''
    finds tag lines in extracted text
    '''
    det_notice_type = None
    for nt_tag in ['Notice Type','NoticeType']:
        n_t_idx = find_element_index(extracted_text,nt_tag)
        if n_t_idx!=[]:
            break

    if n_t_idx!=[]:
        if len(extracted_text)==1:
            for tag in list(all_tags.values()):
                if tag.lower() in extracted_text[n_t_idx].lower():
                    det_notice_type = tag
                    break
        elif len(extracted_text)>1:
            for tag in list(all_tags.values()):
                for tag_index in range(n_t_idx,len(extracted_text)):
                    if tag.lower() in extracted_text[tag_index].lower():
                        det_notice_type = tag
                        break
        else:
            pass

    else:
        if len(extracted_text)==1:
            for tag in list(all_tags.values()):
                det_notice_idx = find_element_index(extracted_text,tag)
                if det_notice_idx!=[]:
                    det_notice_type = tag
                    break
        else:
            for tag in ['Please come to:']:#list(all_tags.values()):
                '''
                this is just to cater "please come to:" forms this phrase appear
                in the top line of the big box
                '''
                det_notice_idx = find_element_index(extracted_text,tag)
                if det_notice_idx!=[] and det_notice_idx==0:
                    det_notice_type = tag
                    break

    if det_notice_type!=None:
        return find_key_by_val(all_tags,det_notice_type)
    else:
        return det_notice_type

def extract_all_tables(img,sp_name,path_cropped,filter_tables=True,save_crops=False,save_overlay=True,save_ext='.jpg',fake_table=True,fake_line=False,resize=2):
    ## https://github.com/arnavdutta/Table-Detection-Extraction/tree/master

    if save_crops==True or save_overlay==True:
        if not os.path.isdir(path_cropped):
            os.mkdir(path_cropped)

    ## Resizing to make text predictions better
    if resize!=None:
        in_img_h,in_img_w = img.shape
        img = cv2.resize(img,(int(in_img_w*resize),int(in_img_h*resize)))


    img_gray = img.copy()
    im_h,im_w = img_gray.shape

    if fake_line==True:
        img_gray = draw_fake_line(img_gray.copy(),line_hei=0.4,line_thick=4)


    ret,img_bin = cv2.threshold(img_gray,220,255,cv2.THRESH_BINARY)
    img_bin = cv2.bitwise_not(img_bin)

    kernel_length_v = (np.array(img_gray).shape[1])//120
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length_v))
    im_temp1 = cv2.erode(img_bin, vertical_kernel, iterations=2)#3
    vertical_lines_img = cv2.dilate(im_temp1, vertical_kernel, iterations=3)

    kernel_length_h = (np.array(img_gray).shape[1])//40
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length_h, 1))
    im_temp2 = cv2.erode(img_bin, horizontal_kernel, iterations=2)#3
    horizontal_lines_img = cv2.dilate(im_temp2, horizontal_kernel, iterations=3)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    table_segment = cv2.addWeighted(vertical_lines_img, 0.5, horizontal_lines_img, 0.5, 0.0)
    table_segment = cv2.erode(cv2.bitwise_not(table_segment), kernel, iterations=2)
    thresh, table_segment = cv2.threshold(table_segment, 0, 255, cv2.THRESH_OTSU)

    contours, hierarchy = cv2.findContours(table_segment, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    overlayed_tables = []
    all_tables = {}
    all_tables['cropped'] = []
    table_coordinates = {}
    table_coordinates['cropped'] = []

    count = 0
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        cropped = img_gray[y:y+h, x:x+w]
        # if (w>80 and h>20) and w>3*h:
        if filter_tables==True: ## in case of resize==2
            if (w>50 and h>20) and (y+h)<(im_h*0.5) and y>(im_h*0.05) and len(np.unique(cropped))!=1 and (w>h):
                count += 1
                cv2.rectangle(img,(x, y),(x + w, y + h),(0, 255, 0), 2)
                all_tables['cropped'].append(cropped)
                table_coordinates['cropped'].append((x, y, w,h, x+w,y+h))

                if save_crops==True:
                    path_crp = os.path.join(path_cropped,sp_name+'_'+'crop_'+str(count)+save_ext)
                    cv2.imwrite(path_crp, cropped)
        else:
            count += 1
            cv2.rectangle(img,(x, y),(x + w, y + h),(0, 255, 0), 2)
            all_tables['cropped'].append(cropped)
            table_coordinates['cropped'].append((x, y, w,h, x+w,y+h))

            if save_crops==True:
                path_crp = os.path.join(path_cropped,sp_name+'_'+'crop_'+str(count)+save_ext)
                cv2.imwrite(path_crp, cropped)

    if save_overlay==True:
        path_overlay = os.path.join(path_cropped,sp_name+'_bb'+save_ext)
        cv2.imwrite(path_overlay, img)
        # cv2.imwrite("/content/tb/table_detect_" + sp_name+save_ext, table_segment)

    all_tables['table_segment'] = table_segment
    all_tables['table_overlayed'] = img

    if fake_table==True:

        all_tables_fake = all_tables.copy()
        table_coordinates_fake = table_coordinates.copy()

        ## fake table
        if table_coordinates['cropped']!=[]:
            fake_table,table_coordinates_fake = make_fake_table(img,table_coordinates_fake,h_limit=0.35,w_limit=0.6)
            all_tables_fake['cropped'].append(fake_table)
            '''Returns fake table with fake coordinates '''
            return all_tables_fake,table_coordinates_fake
    '''Else return original table with original coordinates '''
    return all_tables,table_coordinates


def cropped_tables_to_data(all_tables,table_coordinates,fake_table,notice_types_tables,all_case_types,
                           thresh_filter=False,op_borders=None,display_info=False,op_morph=None,narrow_cell=True):
    name_co = None
    co = None
    # case_type = None
    case_type_tables = None
    notice_type = None

    applicant = None
    petitioner = None
    beneficiary = None
    page_num = None
    for cr in range(0,len(all_tables['cropped'])):
        '''Fake table condition is to limit the finding of only co and name_co in the fake table. As only name and name_co exist in that.
        For everything else do not look into the fake table. It also assumes that fake table is appended at the last index.
        So if fake_table==False look in all the indexes but if its true, do not look into it exept for name and name_co'''
        fake_table_condition = (fake_table==True and cr<len(all_tables['cropped'])-1) or (fake_table==False)
        cropped_img = all_tables['cropped'][cr]

        if min(cropped_img.shape)==0:
            continue

        if narrow_cell==True:
            cropped_contours,eroded_img = erode_and_find_contours(cropped_img,erode_iters=10)
            if display_info==True:
                display_multi(eroded_img)

            if len(cropped_contours)!=0:
                cropped_img = cropped_contours[0]


        crp_h,crp_w = cropped_img.shape

        if op_borders!=None:
            ## add border
            cropped_img = add_image_border(np.array(cropped_img),op_borders,(255,255,255))  ## adding white border to make text not so narroly cropped

        ## testing simple vs otsu thresholding
        # cropped_img = cv2.threshold(cropped_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        if thresh_filter!=None:
            cropped_img = skimage.util.img_as_ubyte(cropped_img > thresh_filter(cropped_img))

        if op_morph!=None:
            cropped_img = op_morph(cropped_img)

        ###extracted_text = pytesseract.image_to_string(cropped_img)
        data_dict = pytesseract.image_to_data(cropped_img, output_type=pytesseract.Output.DICT)
        data_dict = clean_the_data_dict(data_dict,del_list=['',' '])
        extracted_text,all_lines,all_bboxes = text_to_old_format(data_dict)

        if display_info==True:
            print(extracted_text)
            display_multi(cropped_img)


        if extracted_text != '\x0c':
            ##extracted_text = delete_empty_lines(extracted_text)

            if co==None and name_co==None:
                ## for searching c/o and name_co
                cropped_img_thresh,extracted_text_thresh,all_lines_thresh,all_bboxes_thresh = try_with_thresholding(cropped_img)
                co,name_co = find_name_co_v2(extracted_text_thresh,all_lines_thresh,all_bboxes_thresh,dist_limit=30)
                if display_info==True:
                    print(extracted_text_thresh)
                    display_multi(cropped_img_thresh)


            if applicant==None and fake_table_condition==True:
                '''
                think of limiting the len(extracted_text)<=3
                because in some cases there appears "Applicant Name/Email address"
                which you dont want to detect
                in some cases its Applicant then next line name,
                in other case its Applicant then next line some junk id then in third line some name
                '''

                for ap_tag in ['applicant','applica']:
                    indices = find_element_index(extracted_text,ap_tag)
                    if indices!=[] and ('Application'.lower() not in extracted_text[indices].lower()):
                        applicant = extracted_text[-1]
                        break



                applicant = clean_text(applicant)


            if petitioner==None  and fake_table_condition==True:
                indices = find_element_index(extracted_text,"petitioner")
                if indices!=[] and len(extracted_text)!=1:
                    petitioner = extracted_text[-1]
                elif indices!=[] and len(extracted_text)==1:
                    cropped_img_thresh,extracted_text_thresh,all_lines_thresh,all_bboxes_thresh = try_with_thresholding(cropped_img)
                    indices = find_element_index(extracted_text_thresh,"petitioner")
                    petitioner = extracted_text_thresh[-1]
                    if display_info==True:
                        print('From Thresh Image:')
                        print(extracted_text_thresh)
                        display_multi(cropped_img_thresh)

                petitioner = clean_text(petitioner)

            if beneficiary==None  and fake_table_condition==True:
                indices = find_element_index(extracted_text,"beneficiary")
                if indices!=[] and len(extracted_text)!=1:
                    beneficiary = extracted_text[-1]
                elif indices!=[] and len(extracted_text)==1:
                    cropped_img_thresh,extracted_text_thresh,all_lines_thresh,all_bboxes_thresh = try_with_thresholding(cropped_img)
                    indices = find_element_index(extracted_text_thresh,"beneficiary")
                    beneficiary = extracted_text_thresh[-1]
                    if display_info==True:
                        print('From Thresh Image:')
                        print(extracted_text_thresh)
                        display_multi(cropped_img_thresh)

                beneficiary = clean_text(beneficiary)


            if case_type_tables==None  and fake_table_condition==True:
                case_type_tables,case_type_line = find_case_type_from_tables(extracted_text,all_case_types)
                if case_type_tables==None and find_element_index(extracted_text,'Case Type')!=[] or find_element_index(extracted_text,'CaseType')!=[]:
                    cropped_img_thresh,extracted_text_thresh,all_lines_thresh,all_bboxes_thresh = try_with_thresholding(cropped_img)
                    case_type_tables,case_type_line = find_case_type_from_tables(extracted_text_thresh,all_case_types)
                    if display_info==True:
                        print('From Thresh Image:')
                        print(extracted_text_thresh)
                        display_multi(cropped_img_thresh)

            if notice_type==None  and fake_table_condition==True:
                notice_type = find_notice_tags(notice_types_tables,extracted_text)
                if notice_type==None and (find_element_index(extracted_text,"Notice Type")!=[] or find_element_index(extracted_text,"NoticeType")!=[]):
                    '''if notice type is found but text under it is not found(happens sometimes in case of receipt) then try thresholding'''
                    cropped_img_thresh,extracted_text_thresh,all_lines_thresh,all_bboxes_thresh = try_with_thresholding(cropped_img)
                    notice_type = find_notice_tags(notice_types_tables,extracted_text_thresh)

                    if display_info==True:
                        print('From Thresh Image:')
                        print(extracted_text_thresh)
                        display_multi(cropped_img_thresh)


            if page_num==None  and fake_table_condition==True:
                indices = find_element_index_complete(extracted_text,"page")
                if indices!=[]:
                    '''
                    sometimes page number page k agay bhe likha ho ga us ka kia karay ga bc
                    '''
                    page_num = extracted_text[indices+1]
                    page_num = page_num.strip()[0]


    all_tables_info = {}
    all_tables_info['name_co'] = name_co
    all_tables_info['co'] = co
    # all_tables_info['case_type'] = case_type
    all_tables_info['case_type_tables'] = case_type_tables
    all_tables_info['notice_type'] = notice_type

    all_tables_info['applicant'] = applicant
    all_tables_info['petitioner'] = petitioner
    all_tables_info['beneficiary'] = beneficiary

    all_tables_info['page_num'] = page_num

    return all_tables_info

Overwriting table_utils.py


## Common Functions

In [5]:
%%writefile common_functions.py
import cv2
import numpy as np
import skimage
import pytesseract
import re
import math
from pytesseract import Output
from utils.load_and_preprocess_utils import *

def remove_items(lst, value):
    return [item for item in lst if item != value]


def delete_characters(in_string,del_char_list):
    assert (type(del_char_list)==list or del_char_list==None) and (type(in_string)==str)
    # Filter multiple characters from string
    filtered_chars = filter(lambda item: item not in del_char_list, in_string)
    # Join remaining characters in the filtered list
    out_string = ''.join(filtered_chars)

    return out_string

# ## same as delete_characters
# def remove_multiple_characters(in_string,in_remove):
#     return in_string.translate({ord(i): None for i in in_remove})


def find_element_index(in_list,in_str,del_char_list=None,only_first=True):
    assert type(del_char_list)==str or del_char_list==None
    '''
    sees if a string lies in a line or not
    '''
    ## returns only the index of an input string(partial matching)
    ## for now [0]=Returning only the first occurance
    ## keep in mind s.lower()
    if del_char_list==None:
        indices = [i for i, s in enumerate(in_list) if in_str.lower() in s.lower()]
    else:
        indices = [i for i, s in enumerate(in_list) if delete_characters(in_str.lower(),del_char_list.lower()) in s.lower()]

    if indices!=[]:
        if only_first==True:
            indices = indices[0]

    return indices

def find_element_index_complete(in_list,in_str,only_first=True):
    '''
    used for document type as it matches the WHOLE line with a string
    '''
    ## returns only the index of an input string(partial matching)
    ## for now [0]=Returning only the first occurance
    ## keep in mind s.lower()
    indices = [i for i, s in enumerate(in_list) if in_str.lower() == s.lower()]

    if indices!=[]:
        if only_first==True:
            indices = indices[0]

    return indices


def find_key_by_val(in_dict,in_val):
    value = [i for i in in_dict if in_dict[i].lower()==in_val.lower()]
    return value[0]


def delete_empty_lines(extracted_text):
    extracted_text = extracted_text.split('\n')
    extracted_text = list(filter(lambda x: x not in ['',' ','\x0c'], extracted_text))
    return extracted_text

def clean_text(in_text):
    if in_text == None:
        return in_text
    '''
    trying to clean in_text by first spliting the string by ' '(space)
    then removing the entries which have length=1
    '''
    idx_to_del = []
    sp_in_text = in_text.split(' ')
    for i in range(0,len(sp_in_text)):
        if len(sp_in_text[i])==1:
            idx_to_del.append(i)

    new_in_text = del_list_indexes(sp_in_text, idx_to_del)

    clean_in_text = ''
    for i in range(0,len(new_in_text)):
        if new_in_text[i]!='':
            if i<len(new_in_text)-1:
                clean_in_text+=new_in_text[i]+' '
            else:
                clean_in_text+=new_in_text[i]

    return clean_in_text

def delete_list_indices(in_list,unwanted_indices):
    '''
    delete indices from a list
    '''
    for ele in sorted(unwanted_indices, reverse = True):
        del in_list[ele]

    return in_list

def del_list_indexes(re, id_to_del):
    somelist = [i for j, i in enumerate(re) if j not in set(id_to_del)]
    return somelist

def clean_the_data_dict(data_dict,del_list=['',' ']):
    for del_char in del_list:
        empty_indices = find_element_index_complete(data_dict['text'],del_char,only_first=False)
        for k in data_dict:
            data_dict[k] = delete_list_indices(data_dict[k],empty_indices)

    return data_dict

def erode_and_find_contours(img,erode_iters=10):
    ''' for further narrowing the cropped table cell '''
    ''' Specially filtered for name_co wala contour'''
    eroded_img = generate_contour_image(img,iterations=erode_iters)
    e_img_h,e_img_w = eroded_img.shape[:2]
    contours = find_contour_bboxes(eroded_img)
    if len(contours)>1:
        filtered_contours = filter_contour_indices(contours,e_img_h,e_img_w,y_limit=0.4,x_limit=0.4,custom_filters=filter_for_all_contours)
        all_contour_imgs = filter_contours(img,contours,filtered_contours=filtered_contours)
    elif len(contours)==1:
        all_contour_imgs = filter_contours(img,contours,filtered_contours=None)
    else:
        all_contour_imgs = []

    return all_contour_imgs,eroded_img

def filter_for_all_contours(x,y,w,h,y_limit,x_limit,img_h,img_w): ## 50 is in case of resize==2
    return (x<=(img_w*x_limit)) and (w>=h)  and (y<(img_h*y_limit)) and ((y+h)>40) and ((x+w)>50)


def generate_contour_image(img,iterations=10,kernel=np.ones((5,5),np.uint8)):
    ## erode since the image has black text and white bg
    eroded_img = cv2.erode(img,kernel,iterations=iterations)
    eroded_img = cv2.bitwise_not(eroded_img)
    ret, eroded_img = cv2.threshold(eroded_img, 150, 255, cv2.THRESH_BINARY)
    return eroded_img

def find_contour_bboxes(eroded_img):
    contours, hierarchy = cv2.findContours(eroded_img,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    return contours

def crop_contour(img,contours,cont_id):
    x,y,w,h = cv2.boundingRect(contours[cont_id])
    cnt_img = img[y:y+h,x:x+w]
    return cnt_img

def filter_contours(img,contours,filtered_contours=None):
    all_contours = []
    for i in range(0,len(contours)):
        if filtered_contours!=None:
            if i in filtered_contours:
                all_contours.append(crop_contour(img,contours,i))
        else:
            all_contours.append(crop_contour(img,contours,i))

    return all_contours

def filter_contour_indices(contours,img_h,img_w,y_limit,x_limit,custom_filters):
    filtered_contours = []
    for i in range(0,len(contours)):
        x,y,w,h = cv2.boundingRect(contours[i])
        if custom_filters(x,y,w,h,y_limit,x_limit,img_h,img_w)==True:
            filtered_contours.append(i)

    return filtered_contours

def in_filters(x,y,w,h,y_limit,x_limit,img_h,img_w):
    return (x<=(img_w*x_limit)) and (w>=h)  and ((y+h)<(img_h*y_limit))


def calculate_distance(pt_1,pt_2):
    return round(math.dist(pt_1,pt_2))


def find_name_co_v2(all_lines_str,all_lines,all_bboxes,dist_limit=50):
    co_list = None
    name_co = None
    for in_str in ['C/O','c/o','C/o','c/O','clo','c1o']:
        indices = find_element_index(all_lines_str,in_str)
        if indices!=[]:
            ## removing everything except alphanueric and / because / comes in c/o
            co_line = re.sub(r'[^\w\s\/]', '', all_lines_str[indices])

            if co_line.strip().lower()[:3] != in_str.lower():
                ## meaning C/O: doesn't appear at the start of the line
                return None,None
            else:
                bad_dist_indices = check_coordinates_consecutive(all_bboxes,indices,dist_limit=dist_limit)
                co_list = delete_list_indices(all_lines[indices],bad_dist_indices)
                co_list = ' '.join(co_list)

        if co_list!=None:
            ## C/O found
            bad_dist_indices = check_coordinates_consecutive(all_bboxes,indices-1,dist_limit=dist_limit)
            name_co = delete_list_indices(all_lines[indices-1],bad_dist_indices)
            name_co = ' '.join(name_co)
            break

    return co_list,name_co

def text_to_old_format(data_dict):
    one_line = []
    all_lines = []

    bbox_line = []
    all_bboxes = []

    one_line_str = ''
    all_lines_str = []
    for i in range(0,len(data_dict['text'])):
    # for i in tqdm(range(15,40)):
        if data_dict['word_num'][i]==1:
            if i!=0:
                all_lines.append(one_line)
                all_lines_str.append(one_line_str)
                all_bboxes.append(bbox_line)

            one_line = [data_dict['text'][i]]
            one_line_str = data_dict['text'][i]
            bbox_line = [(data_dict['left'][i],data_dict['top'][i],data_dict['width'][i],data_dict['height'][i])]
        else:
            one_line_str = one_line_str+' '+data_dict['text'][i]
            one_line.append(data_dict['text'][i])
            bbox_line.append((data_dict['left'][i],data_dict['top'][i],data_dict['width'][i],data_dict['height'][i]))

    all_lines.append(one_line)
    all_lines_str.append(one_line_str)
    all_bboxes.append(bbox_line)

    return all_lines_str,all_lines,all_bboxes


def try_with_thresholding(cropped_img):
    cropped_img_thresh = skimage.util.img_as_ubyte(cropped_img > skimage.filters.threshold_li(cropped_img))
    data_dict_thresh = pytesseract.image_to_data(cropped_img_thresh, output_type=pytesseract.Output.DICT)
    data_dict_thresh = clean_the_data_dict(data_dict_thresh,del_list=['',' '])
    extracted_text_thresh,all_lines_thresh,all_bboxes_thresh = text_to_old_format(data_dict_thresh)

    return cropped_img_thresh,extracted_text_thresh,all_lines_thresh,all_bboxes_thresh


def check_coordinates_consecutive(all_bboxes,indices,dist_limit=50):
    '''
    measures the distance between the bboxes of detected words
    if distance is > than a threshold then the two words don't belong to same group
    specificaly made for name in front of RE: . E.g. RE: XYZ junk 412-213-
    So sometimes the junk values are also detected as part of name
    '''
    bad_dist_indices = []
    for bb in range(0,len(all_bboxes[indices])-1):
        ## (left,top,width,height)
        lft_1,top_1,wid_1,hei_1 = all_bboxes[indices][bb]
        lft_2,top_2,wid_2,hei_2 = all_bboxes[indices][bb+1]
        bb_dist = calculate_distance((lft_1+wid_1,top_1),(lft_2,top_2))
        if bb_dist>dist_limit:
            bad_dist_indices.append(bb+1)


    return bad_dist_indices


def find_case_type_v2(all_lines_str,all_lines,all_bboxes,all_case_types,in_str,dist_limit=30):
    '''
    finds RE: and then finds case types from case type list. If case type distance <thres with RE then case type is returned
    '''
    (indices_1,in_str_idx_1),(lft_1,top_1,wid_1,hei_1) = get_word_idx_line_pos(all_lines_str,all_lines,all_bboxes,in_str)

    # found_tags = []
    if indices_1!=[]:
        for in_tag in all_case_types:
            try:
                (indices_2,in_str_idx_2),(lft_2,top_2,wid_2,hei_2) = get_word_idx_line_pos(all_lines_str,all_lines,all_bboxes,in_tag)
                if indices_2!=[]:
                    bb_dist = calculate_distance((lft_1,top_1+hei_1),(lft_2,top_2))
                    if bb_dist<dist_limit:
                        return in_tag, all_lines[indices_2][in_str_idx_2]
            except:
                pass

    return None,None

def get_word_idx_line_pos(all_lines_str,all_lines,all_bboxes,in_str):
    '''
    finds the line number and word position of a given string
    '''
    indices = find_element_index(all_lines_str,in_str)
    if indices!=[]:
        in_str_idx = find_element_index(all_lines[indices],in_str)
        lft_1,top_1,wid_1,hei_1 = all_bboxes[indices][in_str_idx]
        return (indices,in_str_idx),(lft_1,top_1,wid_1,hei_1)
    else:
        return None


def find_a_file_num_name(all_lines_str):
    '''In few docs there is A file number: xyz and then in above line there is name of the person '''
    indices = find_element_index(all_lines_str,'A file number')
    if indices!=[] and indices>0:
        return all_lines_str[indices-1]
    else:
        return None

def find_attorney_copy(all_lines_str):
    '''In few docs there is Attorney Copy written '''
    indices = find_element_index(all_lines_str,'Attorney Copy')
    if indices!=[]:
        return True
    else:
        return None


def filters_for_table_detection(x,y,w,h,y_limit,x_limit,img_h,img_w):
    return (w>=(img_w*x_limit)) and (w>=h) and (w<(img_w*0.9))

def filters_for_date_detection(x,y,w,h,y_limit,x_limit,img_h,img_w):
    return (w<=(img_w*x_limit)) and (w>=2*h)

def detect_tables_vs_nontables(img,display_info=False):
    init_type = None

    img = np.array(trim_image_borders(Image.fromarray(img)))

    img_h,img_w = img.shape
    # test_img = img[:int(img_h*0.06),int(img_w*0.5):]
    test_img = img[:int(img_h*0.12),:]
    test_img = skimage.util.img_as_ubyte(test_img > skimage.filters.threshold_otsu(test_img))
    # extracted_text = pytesseract.image_to_string(test_img)
    # extracted_text = delete_empty_lines(extracted_text)

    data_dict = pytesseract.image_to_data(test_img, output_type=Output.DICT)
    data_dict = clean_the_data_dict(data_dict,del_list=['',' '])
    extracted_text,all_lines,all_bboxes = text_to_old_format(data_dict)



    if display_info==True:
        print(' ')
        print('---------------------------------------')
        print('Tables')
    if display_info==True:
        print(extracted_text)
        display_multi(test_img)

    for in_txt in ['Action','Does not grant']:
        indices = find_element_index(extracted_text,in_txt)
        if indices!=[]:
            init_type = 'Tables'
            break

    if init_type==None:
        indices = find_element_index(extracted_text,'IN THE DISTRICT COURT OF APPEAL')
        if indices!=[]:
            init_type = 'Unidentified'

    if init_type==None:
        #####################################################
        ### Search for Normals in tables text too. For cases like oath ceremony and date that appease in a complete line

        for in_txt in ['Oath','Oath Ceremony','Naturalization']:
            indices = find_element_index(extracted_text,in_txt)
            if indices!=[]:
                init_type = 'Normal'
                break

        for in_txt in ['January','February','March','April','May','June','July','August','September','October','November','December']:
            indices = find_element_index(extracted_text,in_txt)
            if indices!=[]:
                bad_dist_indices = check_coordinates_consecutive(all_bboxes,indices,dist_limit=50)

            if indices!=[] and len(extracted_text[indices].split(' '))>5 and bad_dist_indices==[]:
                init_type = 'Useless'
                if display_info==True:
                    print(extracted_text)
                    print(extracted_text[indices])
                    display_multi(test_img)
                break

            if indices!=[] and find_element_index(extracted_text[indices].split(' '),in_txt)!=0:
                init_type = 'Useless'
                if display_info==True:
                    print(extracted_text)
                    print(extracted_text[indices])
                    display_multi(test_img)
                break

    #####################################################
    if init_type==None:
        eroded_img = cv2.erode(test_img,np.ones((1,5),np.uint8),iterations=5)
        eroded_img = cv2.bitwise_not(eroded_img)
        ret, eroded_img = cv2.threshold(eroded_img, 150, 255, cv2.THRESH_BINARY)
        e_img_h,e_img_w = eroded_img.shape[:2]
        contours = find_contour_bboxes(eroded_img)
        filtered_contours = filter_contour_indices(contours,img_h,img_w,y_limit=None,x_limit=0.5,custom_filters=filters_for_table_detection)
        all_contour_imgs = filter_contours(img,contours,filtered_contours=filtered_contours)
        for cnt_img in all_contour_imgs:
            cnt_img = add_image_border(cnt_img,10,255)
            extracted_text = pytesseract.image_to_string(cnt_img)
            extracted_text = delete_empty_lines(extracted_text)
            if display_info==True:
                display_multi(cnt_img)

            for in_txt in ['Action','Does not grant']:
                indices = find_element_index(extracted_text,in_txt)
                if indices!=[]:
                    init_type = 'Tables'
                    break

            if init_type!=None:
                break

    if display_info==True:
        print(' ')
        print('---------------------------------------')
        print('Normal')


    if init_type==None:

        test_img = img[:int(img_h*0.15),:int(img_h*0.25)]
        # test_img = skimage.util.img_as_ubyte(test_img > skimage.filters.threshold_otsu(test_img))
        extracted_text = pytesseract.image_to_string(test_img)
        extracted_text = delete_empty_lines(extracted_text)

        if display_info==True:
            print(extracted_text)
            display_multi(test_img)

        for in_txt in ['January','February','March','April','May','June','July','August','September','October','November','December',
                        'Oath','Oath Ceremony','Naturalization']:
            indices = find_element_index(extracted_text,in_txt)
            if indices!=[] and (in_txt in ['Oath','Oath Ceremony','Naturalization']):
                init_type = 'Normal'
                break

            elif indices!=[] and len(extracted_text[indices].split(' '))<5 and find_element_index(extracted_text[indices].split(' '),in_txt)==0:
                init_type = 'Normal'
                break

    if init_type==None:
        test_img = img[:int(img_h*0.15),:int(img_h*0.25)]
        eroded_img = cv2.erode(test_img,np.ones((3,5),np.uint8),iterations=5)
        eroded_img = cv2.bitwise_not(eroded_img)
        ret, eroded_img = cv2.threshold(eroded_img, 150, 255, cv2.THRESH_BINARY)
        e_img_h,e_img_w = eroded_img.shape[:2]
        contours = find_contour_bboxes(eroded_img)
        filtered_contours = filter_contour_indices(contours,img_h,img_w,y_limit=None,x_limit=0.4,custom_filters=filters_for_date_detection)
        all_contour_imgs = filter_contours(img,contours,filtered_contours=filtered_contours)

        for cnt_img in all_contour_imgs:
            cnt_img = add_image_border(cnt_img,10,255)
            extracted_text = pytesseract.image_to_string(cnt_img)
            extracted_text = delete_empty_lines(extracted_text)
            if display_info==True:
                print(extracted_text)
                display_multi(cnt_img)
            for in_txt in ['January','February','March','April','May','June','July','August','September','October','November','December',
                        'Oath','Oath Ceremony','Naturalization']:
                indices = find_element_index(extracted_text,in_txt)
                if indices!=[] and (in_txt in ['Oath','Oath Ceremony','Naturalization']):
                    init_type = 'Normal'
                    break

                elif indices!=[] and len(extracted_text[indices].split(' '))<5 and find_element_index(extracted_text[indices].split(' '),in_txt)==0:
                    init_type = 'Normal'
                    break

            if init_type!=None:
                break


    # if init_type==None:
    #     init_type = 'None'

    return init_type



Overwriting common_functions.py


## For Non Tables

In [6]:
%%writefile non_table_utils.py

from utils.common_functions import *

import pytesseract
import re
import cv2

import math
from pytesseract import Output
from tqdm import tqdm
import numpy as np
import skimage


def find_re_v2(all_lines_str,all_lines,all_bboxes,in_str='RE:',dist_limit=200,dist_check='consecutive'):
    assert dist_check=='fixed' or dist_check=='consecutive'
    indices = find_element_index(all_lines_str,in_str)

    if indices!=[] and (all_lines_str[indices].strip().lower()[:3] != in_str.lower()):
        ## meaning RE: doesn't appear at the start of the line
        return None

    if indices!=[] and dist_check=='fixed':
        in_str_idx = find_element_index(all_lines[indices],in_str)

    if indices!=[] and dist_check=='fixed':
        bad_dist_indices = check_coordinates_fixed(all_bboxes,indices,in_str_idx,dist_limit=dist_limit)
        ## remove the RE iteself too
        # bad_dist_indices.append(in_str_idx)
        re_list = delete_list_indices(all_lines[indices],bad_dist_indices)
        re_list = ' '.join(re_list)

    elif indices!=[] and dist_check=='consecutive':
        bad_dist_indices = check_coordinates_consecutive(all_bboxes,indices,dist_limit=dist_limit)
        re_list = delete_list_indices(all_lines[indices],bad_dist_indices)
        re_list = ' '.join(re_list)
    else:
        re_list = None

    return re_list


def check_coordinates_fixed(all_bboxes,indices,in_str_idx,dist_limit=200):
    '''
    measures the distance between the bboxes of detected words
    if distance is > than a threshold then the two words don't belong to same group
    specificaly made for name in front of RE: . E.g. RE: XYZ junk 412-213-
    So sometimes the junk values are also detected as part of name
    '''
    lft_1,top_1,wid_1,hei_1 = all_bboxes[indices][in_str_idx]
    bad_dist_indices = []
    for bb in range(0,len(all_bboxes[indices])):
        if bb!=in_str_idx:
            ## (left,top,width,height)
            lft_2,top_2,wid_2,hei_2 = all_bboxes[indices][bb]
            bb_dist = calculate_distance((lft_1+wid_1,top_1),(lft_2,top_2))
            if bb_dist>dist_limit:
                bad_dist_indices.append(bb)


    return bad_dist_indices


def detect_and_extract_document_type(image,config,all_tags,all_tags_partial,adjust_img=True):
    im_h,im_w = image.shape

    if adjust_img==True:
        ## in_image = image[int(im_h*0.25):int(im_h*0.5) , int(im_w*0.0):int(im_w*0.8)] ## adjust for document type
        in_image = image[int(im_h*0.0):int(im_h*0.5) , int(im_w*0.0):int(im_w*0.8)] ## adjust for document type
    else:
        in_image = image

    data_dict = pytesseract.image_to_data(in_image, output_type=pytesseract.Output.DICT)
    data_dict = clean_the_data_dict(data_dict,del_list=['',' '])
    extracted_text,all_lines,all_bboxes = text_to_old_format(data_dict)

    if extracted_text != '\x0c':
        # extracted_text = delete_empty_lines(extracted_text)
        doc_type = match_document_tags(all_tags,all_tags_partial,extracted_text)
    else:
        doc_type = None


    return doc_type,(in_image,extracted_text,all_lines,all_bboxes)

def match_document_tags(all_tags,all_tags_partial,extracted_text):
    '''
    finds tag lines in extracted text
    '''
    for tag in list(all_tags.values()):
        if tag not in all_tags_partial:
            det_idx = find_element_index_complete(extracted_text,tag)
        else:
            det_idx = find_element_index(extracted_text,tag)

        if det_idx!=[]:
            return find_key_by_val(all_tags,tag)

    return None


def find_dear_name(extracted_text):
    '''
    if RE wasn't found then searches for dear NAME (can be seen in NIVCC)
    '''
    det_dear_name = None
    det_idx = find_element_index(extracted_text,'Dear')
    if det_idx!=[]:
        det_dear_name = extracted_text[det_idx].strip('Dear').strip(' ').replace(':','')


    return det_dear_name

def img_to_all_data_dicts(img,do_filtering=True,erode_iters=10):
    # in_img_thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    # in_img_thresh = skimage.filters.threshold_mean(img)
    # in_img_thresh = img > in_img_thresh

    img_h,img_w = img.shape
    all_data_dicts = []

    eroded_img = generate_contour_image(img,iterations=erode_iters)
    contours = find_contour_bboxes(eroded_img)
    if do_filtering==True:
        filtered_contours = filter_contour_indices(contours,img_h,img_w,y_limit=0.6,x_limit=1.0,custom_filters=in_filters)
    else:
        filtered_contours = None

    all_contour_imgs = filter_contours(img,contours,filtered_contours=filtered_contours)

    for cont_idx in range(0,len(all_contour_imgs)):
        in_img_thresh = all_contour_imgs[cont_idx]
        data_dict = pytesseract.image_to_data(in_img_thresh, output_type=Output.DICT)
        data_dict = clean_the_data_dict(data_dict,del_list=['',' '])
        all_lines_str,all_lines,all_bboxes = text_to_old_format(data_dict)
        all_data_dicts.append((all_lines_str,all_lines,all_bboxes))

    return all_data_dicts


def data_dict_to_data(all_data_dicts,all_doc_tags,all_doc_tags_partial,all_case_types):
    doc_type = None
    re = None
    case_type = None
    dear_name = None
    co = None
    name_co = None
    a_file_num_name = None
    attorney_copy = None
    case_type_attorney = None

    all_doc_info = {}
    for data_idx in range(0,len(all_data_dicts)):
        (extracted_text,all_lines,all_bboxes) = all_data_dicts[data_idx]

        if doc_type==None:
            doc_type = match_document_tags(all_doc_tags,all_doc_tags_partial,extracted_text)

        if re==None:
            for re_end in [':',';']:
                re = find_re_v2(extracted_text,all_lines,all_bboxes,in_str='RE'+re_end,dist_check='consecutive',dist_limit=50)## dist_limit depends on image size
                if re!=None:
                    re = re.replace('RE','').strip(re_end).strip()
                    if case_type==None:
                        case_type, case_type_org = find_case_type_v2(extracted_text,all_lines,all_bboxes,all_case_types,'RE'+re_end,dist_limit=30)
                if re!=None:
                    break

        if dear_name==None:
            dear_name = find_dear_name(extracted_text)

        if co==None or name_co==None:
            co,name_co = find_name_co_v2(extracted_text,all_lines,all_bboxes,dist_limit=30)

        if a_file_num_name==None:
            a_file_num_name = find_a_file_num_name(extracted_text)
            if a_file_num_name!=None:
                '''In few cases there is RE: before the name '''
                a_file_num_name = a_file_num_name.strip('RE:')
                a_file_num_name = a_file_num_name.strip('RE;')
                a_file_num_name = a_file_num_name.strip()

        if attorney_copy==None:
            attorney_copy = find_attorney_copy(extracted_text)

        if attorney_copy!=None and case_type_attorney==None:
            for data_idx_atc in range(0,len(all_data_dicts)):
                (extracted_text_atc,all_lines_atc,all_bboxes_atc) = all_data_dicts[data_idx_atc]

                for ct_at in ['Case Type','CaseType']:
                    indices_ct = find_element_index(extracted_text_atc,ct_at)
                    if indices_ct!=[]:
                        for c_t_2 in all_case_types:
                            if c_t_2 in extracted_text_atc[indices_ct]:
                                case_type_attorney = c_t_2
                                break
                    if case_type_attorney!=None:
                        break
                if case_type_attorney!=None:
                        break

    all_doc_info['doc_type'] = doc_type
    all_doc_info['case_type'] = case_type
    all_doc_info['re'] = re
    all_doc_info['dear_name'] = dear_name
    all_doc_info['co'] = co
    all_doc_info['name_co'] = name_co
    all_doc_info['a_file_num_name'] = a_file_num_name
    all_doc_info['attorney_copy'] = attorney_copy
    all_doc_info['case_type_attorney'] = case_type_attorney

    return all_doc_info

Overwriting non_table_utils.py


## Formatting Utils

In [7]:
%%writefile formatting_utils.py

import os
import re
import cv2
from utils.common_functions import delete_characters
import numpy as np
from PyPDF2 import PdfWriter, PdfReader

def format_name(in_name):

    if in_name==None:
        return in_name

    in_name = delete_characters(in_name,[',','.'])

    in_name = in_name.split(' ')
    if len(in_name)>1:
        f_name = in_name[1]+', '+in_name[0]
    else:
        f_name = in_name[0]

    return f_name

def correct_final_doc_type(doc_type):
    ''' if there is "Receipt_2" then make it "Receipt"   if its Decision_Notice then leave it as it is'''
    if doc_type==None or doc_type=='Unidentified':
        return doc_type

    if '_' in doc_type:
        if doc_type.split('_')[1] in ['0','1','2','3','4','5','6','7','8','9','10']:
            doc_type = doc_type.split('_')[0]

    return doc_type

def correct_final_case_type(case_type):

    if case_type!=None:
        case_type = re.sub(r'[^\w\s\/]', '',case_type)
        if case_type=='400':
            case_type = 'N-'+case_type
        elif case_type!='NA' and case_type!='400':
            case_type = 'I-'+case_type
        else:
            case_type = 'NA'

    return case_type

def decide_page(info_dict,info_dict_old,notice_types_tables,all_doc_tags):

    ## tables
    if (info_dict['doc_type']in['Interview','Interview_2']) and (info_dict_old['doc_type']in['Interview','Interview_2']) \
        and (info_dict['case_type']in['485','-485']) and (info_dict_old['case_type']in['485','-485']) \
        and (info_dict['name']==None) and (info_dict_old['name']!=None):
        return True

    elif (info_dict_old['doc_type']in notice_types_tables.keys()) and (info_dict['doc_type']==None) and (info_dict['case_type']==None) and (info_dict['name']==None):
        return True

    elif (info_dict['doc_type']==None) and (info_dict['case_type']==None) and (info_dict['name']==None) and \
        (info_dict_old['doc_type']==None) and (info_dict_old['case_type']==None) and (info_dict_old['name']==None):
        return True

    ## non tables
    elif (info_dict['doc_type']==None) and (info_dict['case_type']==None) and (info_dict['name']==None) and \
        (info_dict_old['doc_type'] in all_doc_tags.keys()) and (info_dict_old['case_type']!=None) and (info_dict_old['name']!=None):
        return True

    elif (info_dict['doc_type']==None) and (info_dict['case_type']==None) and (info_dict['name']==None):
        return True

    else:
        return False


def final_naming_tables(all_tables_info):
    name = None
    case_type = None
    doc_type = None

    ## missing 'USCIS'
    if all_tables_info['notice_type']in['Reopen','Transfer','Approval','Receipt','Receipt_2',
                                        'Rejection','ASC','Interview','Interview_2','Cancellation','Applicants','Biometric']:
        name = all_tables_info['name_co']
        case_type = all_tables_info['case_type_tables']
        doc_type = all_tables_info['notice_type']

    elif all_tables_info['notice_type']=='USCIS':
        name = all_tables_info['applicant']
        case_type = all_tables_info['case_type_tables']
        doc_type = all_tables_info['notice_type']


    if (all_tables_info['name_co']!=None) and \
     ((all_tables_info['name_co'].lower() in 'SOLOMON IMMIGRATION LAW LLC'.lower())  or 'SOLOMON IMMIGRATION'.lower()in  all_tables_info['name_co'].lower()):
        for nm in ['applicant','petitioner']:#'beneficiary'
            if all_tables_info[nm]!=None:
                name = all_tables_info[nm]
                break


    info_dict = {}
    info_dict['name'] = name
    info_dict['case_type'] = case_type
    info_dict['doc_type'] = doc_type
    # info_dict['page'] = 1
    return info_dict


def final_naming_nontables(all_doc_info):
    name = None
    case_type = None
    doc_type = None

    if all_doc_info['init_type']=='Unidentified':
        name = 'Unidentified'
        case_type = 'NA'
        doc_type = 'NA'

    elif (all_doc_info['doc_type']=='RFE') and (all_doc_info['case_type']in['-485','485']):
        name = all_doc_info['re']
        case_type = all_doc_info['case_type']
        doc_type = all_doc_info['doc_type']

    elif (all_doc_info['doc_type']=='RFE') and (all_doc_info['case_type']in['-129F','-130','129F','130']):
        name = all_doc_info['name_co']
        case_type = all_doc_info['case_type']
        doc_type = all_doc_info['doc_type']

    elif (all_doc_info['doc_type']=='Oath_Ceremony'):
        name = all_doc_info['name_co']
        case_type = 'N-400'#all_doc_info['case_type']
        doc_type = all_doc_info['doc_type']

    elif all_doc_info['doc_type'] in ['Deficiency_Notice','Courtesy_letter','Withdrawal','Decision','Decision_Notice','Withdrawal_Acknowledgment']:
        if all_doc_info['name_co']!=None:
            name = all_doc_info['name_co']
        else:
            name = all_doc_info['re']

        case_type = all_doc_info['case_type']
        doc_type = all_doc_info['doc_type']

    elif (all_doc_info['doc_type']=='NIVCC'):
        name = all_doc_info['dear_name']
        case_type = 'NA'#all_doc_info['case_type']
        doc_type = all_doc_info['doc_type']

    elif all_doc_info['doc_type']==None and all_doc_info['a_file_num_name']!=None:
        name = all_doc_info['a_file_num_name']
        case_type = 'NA'
        doc_type = 'NA'

    elif all_doc_info['doc_type']==None and all_doc_info['dear_name']!=None and all_doc_info['case_type_attorney']==None \
            and ['Sir'.lower() not in all_doc_info['dear_name'].lower()] and ['Madam'.lower() not in all_doc_info['dear_name'].lower()]:
        name = all_doc_info['dear_name']
        case_type = 'NA'
        doc_type = 'NA'

    elif all_doc_info['doc_type']==None and all_doc_info['case_type_attorney']!=None:
        name = all_doc_info['dear_name']
        case_type = all_doc_info['case_type_attorney']
        doc_type = 'NA'


    info_dict = {}
    info_dict['name'] = name
    info_dict['case_type'] = case_type
    info_dict['doc_type'] = doc_type
    # info_dict['page'] = 1
    return info_dict


def make_pagewise_list(all_final_names,all_final_appends):
    all_files_pagewise = []
    for i in range(0,len(all_final_appends)):
        one_file = []
        if all_final_appends[i]==False:
            curr_name = all_final_names[i]
            one_file.append((curr_name,i))
            for j in range(i+1,len(all_final_appends)):
                if all_final_appends[j]==True:
                    one_file.append(j)
                else:
                    break

            all_files_pagewise.append(one_file)

    return all_files_pagewise

def break_pdf_to_files(path_in_pdf,path_write_pdf,all_files_pagewise,compress_pdf=False):
    inputpdf = PdfReader(open(path_in_pdf, "rb"))
    for i in range(0,len(all_files_pagewise)):
        one_pdf = all_files_pagewise[i]
        pdf_writer = PdfWriter()
        pdf_name,_ = os.path.splitext(one_pdf[0][0].split('/')[-1])
        pdf_name = os.path.join(path_write_pdf,pdf_name+'.pdf')

        one_page = inputpdf.pages[one_pdf[0][1]]

        if compress_pdf==True:
            one_page.compress_content_streams()

        pdf_writer.add_page(one_page)

        for j in range(1,len(one_pdf)):
            one_page = inputpdf.pages[one_pdf[j]]
            if compress_pdf==True:
                one_page.compress_content_streams()

            pdf_writer.add_page(one_page)

        with open(pdf_name, "wb") as outputStream:
            pdf_writer.write(outputStream)

def break_pdf_to_images(all_pages,path_write_images,all_files_pagewise,jpeg_quality=95):

    for i in range(0,len(all_files_pagewise)):
        one_pdf = all_files_pagewise[i]

        pdf_name,_ = os.path.splitext(one_pdf[0][0].split('/')[-1])
        pdf_name = os.path.join(path_write_images,pdf_name+'.jpg')

        one_page = np.array(all_pages[one_pdf[0][1]])

        final_img = np.array(one_page)

        for j in range(1,len(one_pdf)):
            one_page = all_pages[one_pdf[j]]
            final_img = np.concatenate((final_img,one_page),axis=1)


        cv2.imwrite(pdf_name,final_img,[cv2.IMWRITE_JPEG_QUALITY, jpeg_quality])

Overwriting formatting_utils.py


# Pipeline

In [8]:
%cd '/content/pdf_parsing/'

/content/pdf_parsing


In [9]:
from pdf2image import convert_from_path
# from google.colab import files
import pytesseract
from PIL import Image
import os
import shutil
import re
import cv2

import math
# from pytesseract import Output

from tqdm import tqdm
from glob import glob

# for de skew
# from deskew import determine_skew  ## gray image
import numpy as np
import skimage
# from skimage.transform import rotate

from PyPDF2 import PdfWriter, PdfReader


# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
# def cv2_imshow_rgb(img,resize=None):
#     if resize!=None:
#         img=cv2.resize(img,resize)


#     cv2_imshow(cv2.cvtColor(np.uint8(img),cv2.COLOR_RGB2BGR))

def cv2_imshow_rgb(img,resize=None, figsize=(15,15)):
    if resize!=None:
        img=cv2.resize(img,resize)

        #     cv2_imshow(cv2.cvtColor(np.uint8(img),cv2.COLOR_RGB2BGR))

    plt.figure(figsize=figsize)
    plt.imshow(np.uint8(img))


def display_multi(*images,resize=None, figsize=(15,15),bgr=False,axis=1):
    if resize!=None:
        res = np.array(cv2.resize(images[0],resize))
    else:
        res = np.array(images[0])

    for i in range(1,len(images)):

        if resize!=None:
            res_img = np.array(cv2.resize(images[i],resize))
        else:
            res_img = np.array(images[i])

        res = np.concatenate((res, res_img), axis=axis)

    if bgr==True:
        res = cv2.cvtColor(res,cv2.COLOR_BGR2RGB)

    return cv2_imshow_rgb(res,resize=None, figsize=figsize)

In [10]:
from utils.load_and_preprocess_utils import preprocess_image
from utils.table_utils import extract_all_tables,cropped_tables_to_data
from utils.non_table_utils import img_to_all_data_dicts,data_dict_to_data
from utils.non_table_utils import erode_and_find_contours
from utils.formatting_utils import *
from utils.notice_and_case_types import *
from utils.common_functions import *

In [11]:
def process_and_save(all_pages,junk_folder,display_info=False,st_i=None,end_i=None,save_jpegs=False):
    ## Settings

    ## pre processing options
    op_dpi = (300,300)
    op_deskew = True
    op_img_thresh = False   ## True makes I to 1 so keep it false
    op_borders = (False,10,(0,0,0))

    ## thresholding and morphology
    # ['try_all_threshold','threshold_otsu','threshold_yen','threshold_isodata','threshold_li','threshold_local','threshold_minimum','threshold_mean', 'threshold_niblack','threshold_sauvola','threshold_triangle','apply_hysteresis_threshold','threshold_multiotsu']
    thresh_filter = None#skimage.filters.threshold_otsu
    op_morph = None #apply_morph(img,morph_operation=cv2.MORPH_CLOSE,kernel=(3,3),iterations=1)


    all_final_names = []
    all_final_appends = []
    ## Init dictionaries
    all_tables_info = {}
    all_doc_info = {}

    info_dict_old = {}
    info_dict_old['name'] = None
    info_dict_old['case_type'] = None
    info_dict_old['doc_type'] = None

    info_dict = {}
    info_dict['name'] = None
    info_dict['case_type'] = None
    info_dict['doc_type'] = None

    corrupts = []
    if st_i==None:
        st_i = 0
    if end_i==None:
        end_i = len(all_pages)

    for i in tqdm(range(st_i,end_i)):
        # try:
        pg_num = i+1

        ## Init dictionaries
        all_tables_info = {}
        all_doc_info = {}
        info_dict = {}

        sp_name = 'page_'+str(pg_num)
        img = cv2.cvtColor(np.array(all_pages[i]),cv2.COLOR_RGB2GRAY)

        try:
            img = check_orientation_tesseract(np.array(img))
        except:
            pass

        img = preprocess_image(img,deskew=op_deskew,img_thresh=op_img_thresh,\
                                add_border=op_borders[0],border_width=op_borders[1],border_color=op_borders[2],img_type='gray')

        img_h,img_w = img.shape

        init_type = detect_tables_vs_nontables(img,display_info=False)
        if display_info==True:
            print('')
            print('Table/Not: ',init_type)

        ## ------------------------------------------
        ### Detection using tables
        ## ------------------------------------------
        ## resize image to make sure the capital letters are 30pixels in height
        if init_type=='Tables':
            table_resize = 2
            fake_table = True
            all_tables,table_coordinates = extract_all_tables(img.copy(),sp_name,path_cropped="page_tables/",filter_tables=True,
                                                            save_crops=False,save_overlay=False,fake_table=fake_table,fake_line=False,
                                                            resize=table_resize)


            narrow_cell = True
            all_tables_info = cropped_tables_to_data(all_tables,table_coordinates,fake_table,notice_types_tables,all_case_types_tables,thresh_filter=thresh_filter,
                                        op_borders=10,display_info=display_info,op_morph=op_morph,narrow_cell=narrow_cell)

            all_tables_info['init_type'] = init_type
            # if all_tables_info['notice_type'] in notice_types_tables.keys():
            info_dict = final_naming_tables(all_tables_info)

        else:
            ## ------------------------------------------
            ### Detection without tables
            ## ------------------------------------------
            all_data_dicts = img_to_all_data_dicts(img)
            all_doc_info = data_dict_to_data(all_data_dicts,all_doc_tags,all_doc_tags_partial,all_case_types_tables)
            all_doc_info['init_type'] = init_type
            info_dict = final_naming_nontables(all_doc_info)

            if display_info==True:
                print('----------------------')
                print('All data dicts')
                print('----------------------')
                for d_d in all_data_dicts:
                    print(d_d[0])

        append_page = decide_page(info_dict,info_dict_old,notice_types_tables,all_doc_tags)
        ## update old dict
        info_dict_old = info_dict

        ## Extract important info
        final_name = info_dict['name']
        case_type = info_dict['case_type']
        doc_type = info_dict['doc_type']

        ## Correct final formats
        final_name = format_name(final_name)
        case_type = correct_final_case_type(case_type)
        doc_type = correct_final_doc_type(doc_type)

        ##############################################
        ## saving images

        save_name_full = None

        save_name_full = str(pg_num)+'_'+str(final_name)+'_'+str(case_type)+'_'+str(doc_type)+".jpg"
        save_name_full = os.path.join(junk_folder,save_name_full)
        if save_jpegs==True:
            cv2.imwrite(save_name_full,img,[cv2.IMWRITE_JPEG_QUALITY, 10])

        all_final_names.append(save_name_full)
        all_final_appends.append(append_page)

        if display_info==True:
            print(' ')
            print(save_name_full)
            print('Page:',append_page)
            # print(all_tables_info)
            print('-----------Tables-------------------')
            for k in all_tables_info:
                print(k,'==',all_tables_info[k])

            print('-----------No Tables-------------------')
            for k in all_doc_info:
                print(k,'==',all_doc_info[k])

            print('-----------Info Dict-------------------')
            for k in info_dict:
                print(k,'==',info_dict[k])

            display_multi(img)

        # except:
        #     print('')
        #     print('Corrupt:',pdf_name,pg_num)
        #     corrupts.append((pdf_name,pg_num))

    return all_final_names,all_final_appends,corrupts

In [12]:
glob('/content/*.png')

[]

In [13]:
# all_documents = glob("/content/documents/forms/*")
all_documents = glob("/content/documents/AI Receipts Project/*")

print('Total Documents:',len(all_documents))
print(all_documents)

pdf_path = '/content/documents/AI Receipts Project/Z-1.pdf' #all_documents[1]
pdf_name,_ = os.path.splitext(pdf_path.split('/')[-1])
print(pdf_path,pdf_name)
print('Size(Mb):',round(os.path.getsize(pdf_path)/1024/1024))

## convert pdf to images
all_pages = convert_from_path(pdf_path)
print('Total Pages:',len(all_pages))



Total Documents: 40
['/content/documents/AI Receipts Project/D-1.pdf', '/content/documents/AI Receipts Project/H-2.pdf', '/content/documents/AI Receipts Project/B-2.pdf', '/content/documents/AI Receipts Project/K-1.pdf', '/content/documents/AI Receipts Project/L-1.pdf', '/content/documents/AI Receipts Project/O-1.pdf', '/content/documents/AI Receipts Project/P-2.pdf', '/content/documents/AI Receipts Project/H-1.pdf', '/content/documents/AI Receipts Project/B-1.pdf', '/content/documents/AI Receipts Project/U-1.pdf', '/content/documents/AI Receipts Project/V-2.pdf', '/content/documents/AI Receipts Project/I-1.pdf', '/content/documents/AI Receipts Project/A-2.pdf', '/content/documents/AI Receipts Project/M-2.pdf', '/content/documents/AI Receipts Project/G-2.pdf', '/content/documents/AI Receipts Project/E-1.pdf', '/content/documents/AI Receipts Project/T-1.pdf', '/content/documents/AI Receipts Project/K-2.pdf', '/content/documents/AI Receipts Project/M-1.pdf', '/content/documents/AI Receip

In [14]:
junk_folder = os.path.join('/content/',pdf_name+'_junk/')
!rm -r {junk_folder}

if not os.path.isdir(junk_folder):
    os.mkdir(junk_folder)

# F2-18
# S1-19

st_i = 0
num_forms = 5
end_i = st_i+num_forms
# end_i = len(all_pages)#st_i+num_forms #min(st_i+num_forms,len(selected_pages))

# all_pages = [Image.open('/content/K1-3.png')]
selected_pages = all_pages

display_info=False
save_jpegs = True
all_final_names,all_final_appends,corrupts = process_and_save(all_pages,junk_folder,display_info=display_info,st_i=st_i,end_i=end_i,save_jpegs=save_jpegs)
# display_multi(all_pages[st_i])


100%|██████████| 5/5 [01:04<00:00, 12.91s/it]


In [15]:
path_write_pdf = os.path.join('/content/',pdf_name+'_pdfs/')
path_write_images = os.path.join('/content/',pdf_name+'_images/')

!rm -r {path_write_pdf}
!rm -r {path_write_images}

if not os.path.isdir(path_write_pdf):
    os.mkdir(path_write_pdf)


if not os.path.isdir(path_write_images):
    os.mkdir(path_write_images)

all_files_pagewise = make_pagewise_list(all_final_names,all_final_appends)

break_pdf_to_files(pdf_path,path_write_pdf,all_files_pagewise,compress_pdf=True)

selected_pages = selected_pages[st_i:end_i]
break_pdf_to_images(selected_pages,path_write_images,all_files_pagewise,jpeg_quality=10)

zip_name = os.path.join('/content/',pdf_name+'.zip')
!zip -r {zip_name} {path_write_images}


updating: content/Z-1_images/ (stored 0%)
updating: content/Z-1_images/5_ALBERTO, BRIANT_I-485_ASC.jpg (deflated 26%)
updating: content/Z-1_images/4_A, BRIANT_I-485_Receipt.jpg (deflated 41%)
updating: content/Z-1_images/3_A, BRIANT_I-131_Receipt.jpg (deflated 42%)
updating: content/Z-1_images/2_A, BRIANT_I-765_Receipt.jpg (deflated 23%)
updating: content/Z-1_images/1_CRAIG, ZELL_I-130_Approval.jpg (deflated 24%)


In [ ]:
# all_documents = glob("/content/documents/forms/*")
all_documents = glob("/content/documents/AI Receipts Project/*")

print('Total Documents:',len(all_documents))
print(all_documents)

pdf_path = '/content/documents/AI Receipts Project/A-2.pdf' #all_documents[1]
pdf_name,_ = os.path.splitext(pdf_path.split('/')[-1])
print(pdf_path,pdf_name)
print('Size(Mb):',round(os.path.getsize(pdf_path)/1024/1024))

## convert pdf to images
all_pages = convert_from_path(pdf_path)
print('Total Pages:',len(all_pages))


# # pdf_path = '/content/documents/131_raw/'
# all_pages = glob(os.path.join(pdf_path,'*'))
# pdf_name,_ = os.path.splitext(pdf_path.split('/')[-1])
print(pdf_path,pdf_name)

In [ ]:
!rm -r '/content/tables/'
path_save_types = '/content/tables/'
if not os.path.isdir(path_save_types):
    os.mkdir(path_save_types)


## pre processing options
op_dpi = (300,300)
op_deskew = True
op_img_thresh = False   ## True makes I to 1 so keep it false
op_borders = (False,10,(0,0,0))

## thresholding and morphology
# ['try_all_threshold','threshold_otsu','threshold_yen','threshold_isodata','threshold_li','threshold_local','threshold_minimum','threshold_mean', 'threshold_niblack','threshold_sauvola','threshold_triangle','apply_hysteresis_threshold','threshold_multiotsu']
thresh_filter = None#skimage.filters.threshold_otsu
op_morph = None #apply_morph(img,morph_operation=cv2.MORPH_CLOSE,kernel=(3,3),iterations=1)

selected_pages = all_pages.copy()
st_i = 9
num_forms = 1
end_i = st_i+num_forms
# end_i = len(selected_pages)#st_i+num_forms #min(st_i+num_forms,len(selected_pages))

### selected_pages = selected_pages[st_i:end_i]
print('start/end',st_i,end_i)
display_info=True

for j in range(st_i,end_i):
    # img = cv2.imread(all_pages[j],0)
    img = cv2.cvtColor(np.array(all_pages[j-1]),cv2.COLOR_RGB2GRAY)

    try:
        img = check_orientation_tesseract(np.array(img))
    except:
        pass

    img = preprocess_image(img,deskew=op_deskew,img_thresh=op_img_thresh,\
                            add_border=op_borders[0],border_width=op_borders[1],border_color=op_borders[2],img_type='gray')

    img_h,img_w = img.shape
    init_type = detect_tables_vs_nontables(img,display_info=display_info)

    save_img_name = os.path.join(path_write_images,pdf_name,str(j+1)+'_page_'+init_type+'.jpg')
    cv2.imwrite(save_img_name,np.array(all_pages[j]),[cv2.IMWRITE_JPEG_QUALITY, 10])

print(init_type)
display_multi(img)

In [ ]:
## pre processing options
op_dpi = (300,300)
op_deskew = True
op_img_thresh = False   ## True makes I to 1 so keep it false
op_borders = (False,10,(0,0,0))

## thresholding and morphology
# ['try_all_threshold','threshold_otsu','threshold_yen','threshold_isodata','threshold_li','threshold_local','threshold_minimum','threshold_mean', 'threshold_niblack','threshold_sauvola','threshold_triangle','apply_hysteresis_threshold','threshold_multiotsu']
thresh_filter = None#skimage.filters.threshold_otsu
op_morph = None #apply_morph(img,morph_operation=cv2.MORPH_CLOSE,kernel=(3,3),iterations=1)

# all_documents = glob("/content/documents/forms/*")
all_documents = glob("/content/documents/AI Receipts Project/*")
all_documents.sort()
print('Total Documents:',len(all_documents))
print(all_documents)

!rm -r '/content/all_pdf_tables/'


display_info=False
save_jpegs = True
st_i = None
num_forms = 1
end_i = None#st_i+num_forms

path_write_images = '/content/all_pdf_tables/'
if not os.path.isdir(path_write_images):
    os.mkdir(path_write_images)

all_corrupts = []
for doc_index in tqdm(range(0,len(all_documents))):

    pdf_path = all_documents[doc_index]
    pdf_name,_ = os.path.splitext(pdf_path.split('/')[-1])
    if not os.path.isdir(os.path.join(path_write_images,pdf_name)):
        os.mkdir(os.path.join(path_write_images,pdf_name))

    print(pdf_path,pdf_name)
    print('Size(Mb):',round(os.path.getsize(pdf_path)/1024/1024))

    ## convert pdf to images
    all_pages = convert_from_path(pdf_path)
    print('Total Pages:',len(all_pages))


    for j in tqdm(range(0,len(all_pages))):
        # img = cv2.imread(all_pages[j],0)
        img = cv2.cvtColor(np.array(all_pages[j]),cv2.COLOR_RGB2GRAY)

        # try:
        #     img = check_orientation_tesseract(np.array(img))
        # except:
        #     pass

        img = preprocess_image(img,deskew=op_deskew,img_thresh=op_img_thresh,\
                                add_border=op_borders[0],border_width=op_borders[1],border_color=op_borders[2],img_type='gray')

        img_h,img_w = img.shape
        init_type = detect_tables_vs_nontables(img,display_info=display_info)

        save_img_name = os.path.join(path_write_images,pdf_name,str(j+1)+'_page_'+str(init_type)+'.jpg')
        cv2.imwrite(save_img_name,np.array(all_pages[j]),[cv2.IMWRITE_JPEG_QUALITY, 10])

    # zip_name = os.path.join(save_zip_path,pdf_name+'.zip')
    # !zip -r {zip_name} {path_write_images}

In [ ]:
!zip -r '/content/all_pdf_tables.zip' '/content/all_pdf_tables/'
!cp '/content/all_pdf_tables.zip' '/content/drive/MyDrive/immediate/'

In [ ]:
# print(corrupts)

# [('A-2', 1), ('D-1', 3), ('D-1', 6), ('H-1', 31), ('P-2', 44), ('R-1', 39), ('V-1', 15)]

In [ ]:
# B-2 3 what form is this
# K-1 3  what form is this
# M-2 62,69 what form is this
# P-1 30 ma masla ha. pichlay waloon k saat append hua wa tha
# P-2 20 what form is this
# R-1 24 what form is this
# S-1 19 "Biometric Notification"
# V-1 3  "Biometric Notification"


In [ ]:
# junk 5
# C-2 45-46-47-48 should have 4 pages
# S-1 completely bad

In [ ]:
## if name==None, case=485, doc_type=Interview   THEN page=2
## if prevous doc_type=RFE and current name,doc_type,case_type=None THEN page=2
## if previous=None,None,None and current=None,None,None then page=3

In [ ]:
## Tables vs non tables
# B2-3
# J2-7
# K1-3
# M1-19
# M2-40
# P1-28
# P2-20
# R1-24
# V2-16


# F2-18
# S1-19

In [ ]:
sad

# Loop

In [ ]:
# all_documents = glob("/content/documents/forms/*")
all_documents = glob("/content/documents/AI Receipts Project/*")
all_documents.sort()
print('Total Documents:',len(all_documents))
print(all_documents)


In [ ]:

## Settings

## pre processing options
op_dpi = (300,300)
op_deskew = True
op_img_thresh = False   ## True makes I to 1 so keep it false
op_borders = (False,10,(0,0,0))

## thresholding and morphology
# ['try_all_threshold','threshold_otsu','threshold_yen','threshold_isodata','threshold_li','threshold_local','threshold_minimum','threshold_mean', 'threshold_niblack','threshold_sauvola','threshold_triangle','apply_hysteresis_threshold','threshold_multiotsu']
thresh_filter = None#skimage.filters.threshold_otsu
op_morph = None #apply_morph(img,morph_operation=cv2.MORPH_CLOSE,kernel=(3,3),iterations=1)


In [ ]:
# path_write_pdf = os.path.join('/content/',pdf_name+'_pdfs/')
# path_write_images = os.path.join('/content/',pdf_name+'_images/')

# !rm -r {path_write_pdf}
# !rm -r {path_write_images}

# if not os.path.isdir(path_write_pdf):
#     os.mkdir(path_write_pdf)


# if not os.path.isdir(path_write_images):
#     os.mkdir(path_write_images)

In [ ]:
# !rm -r '/content/drive/MyDrive/micheal_junk_7/'
!mkdir '/content/drive/MyDrive/micheal_junk_8/'
save_zip_path = '/content/drive/MyDrive/micheal_junk_8/'

In [ ]:
display_info=False
save_jpegs = True
st_i = None
num_forms = 1
end_i = None#st_i+num_forms

all_corrupts = []
for doc_index in tqdm(range(0,len(all_documents))):

    pdf_path = all_documents[doc_index]
    pdf_name,_ = os.path.splitext(pdf_path.split('/')[-1])
    print(pdf_path,pdf_name)
    print('Size(Mb):',round(os.path.getsize(pdf_path)/1024/1024))

    if os.path.isfile(os.path.join(save_zip_path,pdf_name+'.zip')):
        continue

    ## convert pdf to images
    all_pages = convert_from_path(pdf_path)
    print('Total Pages:',len(all_pages))

    junk_folder = os.path.join('/content/',pdf_name+'_junk/')
    !rm -r {junk_folder}
    if not os.path.isdir(junk_folder):
        os.mkdir(junk_folder)

    path_write_pdf = os.path.join('/content/',pdf_name+'_pdfs/')
    path_write_images = os.path.join('/content/',pdf_name+'_images/')

    !rm -r {path_write_pdf}
    !rm -r {path_write_images}

    if not os.path.isdir(path_write_pdf):
        os.mkdir(path_write_pdf)


    if not os.path.isdir(path_write_images):
        os.mkdir(path_write_images)

    all_final_names,all_final_appends,corrupts = process_and_save(all_pages,junk_folder,display_info=display_info,st_i=st_i,end_i=end_i,save_jpegs=save_jpegs)
    all_corrupts.append(all_corrupts)

    all_files_pagewise = make_pagewise_list(all_final_names,all_final_appends)
    break_pdf_to_files(pdf_path,path_write_pdf,all_files_pagewise,compress_pdf=True)

    selected_pages = all_pages[st_i:end_i]
    break_pdf_to_images(selected_pages,path_write_images,all_files_pagewise,jpeg_quality=10)

    zip_name = os.path.join(save_zip_path,pdf_name+'.zip')
    !zip -r {zip_name} {path_write_images}


In [ ]:
print(all_corrupts)

In [ ]:
# !mkdir '/content/drive/MyDrive/micheal_junk_5/'

In [ ]:

# print(corrupts)

# [('A-2', 1), ('D-1', 3), ('D-1', 6), ('H-1', 31), ('P-2', 44), ('R-1', 39), ('V-1', 15)]

In [ ]:
# !mkdir '/content/drive/MyDrive/micheal_junk_2/'

In [ ]:
# # all_documents = glob("/content/documents/forms/*")
# all_documents = glob("/content/documents/AI Receipts Project/*")
# all_documents.sort()
# print('Total Documents:',len(all_documents))
# print(all_documents)

# !rm -r '/content/all_pdf_images/'
# display_info=False
# save_jpegs = True
# st_i = None
# num_forms = 1
# end_i = None#st_i+num_forms

# path_write_images = '/content/all_pdf_images/'
# if not os.path.isdir(path_write_images):
#     os.mkdir(path_write_images)

# all_corrupts = []
# for doc_index in tqdm(range(0,len(all_documents))):

#     pdf_path = all_documents[doc_index]
#     pdf_name,_ = os.path.splitext(pdf_path.split('/')[-1])
#     if not os.path.isdir(os.path.join(path_write_images,pdf_name)):
#         os.mkdir(os.path.join(path_write_images,pdf_name))

#     print(pdf_path,pdf_name)
#     print('Size(Mb):',round(os.path.getsize(pdf_path)/1024/1024))

#     ## convert pdf to images
#     all_pages = convert_from_path(pdf_path)
#     print('Total Pages:',len(all_pages))


#     for j in range(0,len(all_pages)):
#         save_img_name = os.path.join(path_write_images,pdf_name,'page_'+str(j+1)+'.jpg')
#         cv2.imwrite(save_img_name,np.array(all_pages[j]),[cv2.IMWRITE_JPEG_QUALITY, 10])

# Split document

In [ ]:
# !pip install PyPDF2

# !mkdir /content/ddd
# from PyPDF2 import PdfWriter, PdfReader

# inputpdf = PdfReader(open('/content/documents/forms/Michael Forms.pdf', "rb"))

# for i in range(len(inputpdf.pages)):
#     output = PdfWriter()
#     output.add_page(inputpdf.pages[i])
#     with open("/content/ddd/document-page%s.pdf" % i, "wb") as outputStream:
#         output.write(outputStream)

#Notes

In [ ]:
## Notes
# deskew
# dpi 300 best
# border/no-border

# Test
# test thresholding
# load_system_dawg and load_freq_dawg to false
# The text detection for tables is image resolution*2 and for normal document is resolution*1. remember 30pix is ideal case


In [ ]:
# Ideas
# Find just the table or its entries and read the values directly for example the cell with petitioner should have a have directly below it
# Delete the unnecessery columns
# Read the document only optill a certain point
# Put black color on the unnecessery areas
# .tiff format for printing


In [ ]:
## explanation for psm types
# yaheen sa manay --psm 4 for tables dakha tha
# https://pyimagesearch.com/2021/11/15/tesseract-page-segmentation-modes-psms-explained-how-to-improve-your-ocr-accuracy/


## detecting tables from images
# https://betterprogramming.pub/extracting-tables-from-images-in-python-made-easy-ier-3be959555f6f

In [ ]:
## for table detection
'''
Note: Depending on the image, you may have to modify the kernel size. For instance to capture longer horizontal lines,
it may be necessary to increase the horizontal kernel from (40, 1) to say (80, 1). If you wanted to detect thicker horizontal lines,
then you could increase the width of the kernel to say (80, 2). In addition, you could increase the number of iterations when performing cv2.morphologyEx().
Similarly, you could modify the vertical kernels to detect more or less vertical lines.
There is a trade-off when increasing or decreasing the kernel size as you may capture more or less of the lines. Again, it all varies depending on the input image
'''

In [ ]:
## For case type detection
'''
make sure if all case types are I-something. If thats the case the just hadcode I and then extract the rest of it.
some case dont have a "-" in I-123  see how to tackle that?
'''

In [ ]:
## utilize str.strip() and str.replace()